In [24]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

start_date = '2022-01-01'
today_date = datetime.today().strftime('%Y-%m-%d')
date_range = pd.date_range(start=start_date, end=today_date).strftime('%Y-%m-%d')
print(f"the data span from {date_range.min()} up to {date_range.max()}")

the data span from 2022-01-01 up to 2024-12-21


In [25]:
fed_rates = pd.read_excel('data/fed_eff_rate_2010-24.xlsx',sheet_name='Daily, 7-Day')
fed_rates.columns = ['date','rate']
fed_rates['date'] = pd.to_datetime(fed_rates['date'])
fed_rates['rate_change'] = fed_rates['rate'].diff()
fed_rates.head()
fed_rates = fed_rates.query('date > @start_date')

In [26]:
df = yf.download('ZF=F', start=start_date)

[*********************100%***********************]  1 of 1 completed


In [27]:
df.reset_index(inplace=True)
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-01-03,120.875000,120.890625,120.421875,120.437500,120.437500,907964
1,2022-01-04,120.507812,120.546875,120.320312,120.429688,120.429688,949320
2,2022-01-05,120.500000,120.539062,120.085938,120.117188,120.117188,1016140
3,2022-01-06,120.132812,120.226562,119.843750,119.867188,119.867188,1078231
4,2022-01-07,119.898438,119.976562,119.632812,119.734375,119.734375,1102805


In [31]:
df['body'] = df['Close'] - df['Open']
df['high_low'] = df['High'] - df['Low']

# Calculate daily percentage change in close price
df['Pct_Change'] = df['Close'].pct_change() * 100  # Multiply by 100 for percentage
df['Absolute_Change'] = df['Close'].diff()

# Define thresholds for "unusual" changes
std = 2
mean_change = df['Pct_Change'].mean()
std_dev_change = df['Pct_Change'].std()
upper_bound = mean_change + std * std_dev_change
lower_bound = mean_change - std * std_dev_change

# Flag unusual days
df['Unusual'] = (df['Pct_Change'] > upper_bound) | (df['Pct_Change'] < lower_bound)


In [32]:
px.line(x=df['Date'].tail(30),y=df['Close'].tail(30).diff())

In [33]:

import plotly.graph_objects as go


fig = go.Figure()

# Add the line plot for percentage change
fig.add_trace(go.Scatter(
    x=df['Date'], y=df['Pct_Change'],
    mode='lines', name='Daily % Change',
    line=dict(color='blue')
))

# Add the upper and lower bounds
fig.add_trace(go.Scatter(
    x=df['Date'], y=[upper_bound] * len(df),
    mode='lines', name=f'Upper Bound ({std} Std Dev)',
    line=dict(color='red', dash='dash')
))
fig.add_trace(go.Scatter(
    x=df['Date'], y=[lower_bound] * len(df),
    mode='lines', name=f'Lower Bound ({std} Std Dev)',
    line=dict(color='green', dash='dash')
))

# Highlight unusual points
fig.add_trace(go.Scatter(
    x=df.loc[df['Unusual'], 'Date'],
    y=df.loc[df['Unusual'], 'Pct_Change'],
    mode='markers', name='Unusual Changes',
    marker=dict(color='orange', size=10, symbol='circle')
))

# Add interest rate changes
fig.add_trace(go.Scatter(
    x=fed_rates['date'], y=fed_rates['rate_change'],
    mode='lines+markers', name='Interest Rate Change',
    line=dict(color='purple'),
    yaxis='y2'  # Use the secondary y-axis
))

fig.update_layout(
    title="Daily Percentage Change in Stock Price and Interest Rate Changes",
    xaxis_title="Date",
    yaxis_title="Percentage Change (%)",
    yaxis2=dict(
        title="Interest Rate Change (%)",
        overlaying='y',
        side='right'
    ),
    legend_title="Legend",
    template="plotly_white"
)

# Show plot
fig.show()


In [40]:
import plotly.graph_objects as go

fig = go.Figure()

# Add the line plot for ZF percentage change
fig.add_trace(go.Scatter(
    x=df['Date'], y=df['Close'],
    mode='lines', name='ZF Daily % Change',
    line=dict(color='blue')
))

# Add the line plot for Federal Reserve rates
fig.add_trace(go.Scatter(
    x=fed_rates['date'], y=fed_rates['rate'],
    mode='lines', name='Fed Rates',
    line=dict(color='purple'),
    yaxis='y2'  # Use the secondary y-axis
))

# Update layout for dual y-axes
fig.update_layout(
    title="ZF close price vs fed effective rate",
    xaxis_title="Date",
    yaxis=dict(
        title="ZF Percentage Change (%)",
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue')
    ),
    yaxis2=dict(
        title="Federal Reserve Rates (%)",
        titlefont=dict(color='purple'),
        tickfont=dict(color='purple'),
        overlaying='y',
        side='right'
    ),
    legend_title="Legend",
    template="plotly_white"
)

# Show plot
fig.show()
